In [1]:
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np

train_df = pd.read_csv("./split-data/train_new.csv")
test_df = pd.read_csv("./split-data/test_new.csv")
val_df = pd.read_csv("./split-data/val_new.csv")

In [3]:
traingen=ImageDataGenerator(rescale= 1./255)
train_generator=traingen.flow_from_dataframe(
dataframe=train_df,
directory="./dataset",
x_col="img_path",
y_col= "category",
batch_size=128,
shuffle= True,
target_size = (256,256),
class_mode="categorical",
workers=4
)

valgen = ImageDataGenerator(rescale=1./255)
val_generator = valgen.flow_from_dataframe(
    dataframe=val_df,
    directory="./dataset",
    x_col="img_path",
    y_col="category",
    target_size=(256, 256),
    batch_size=128,
    class_mode="categorical",
    workers=4
)

Found 4011 validated image filenames belonging to 46 classes.
Found 3139 validated image filenames belonging to 46 classes.


In [4]:
from keras.models import Model
from keras.layers import Dense
from keras.regularizers import l2
from keras.optimizers import SGD,Adam
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import DirectoryIterator, ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard
from keras import backend as K
from keras.layers import Dense, Flatten
from tensorflow.keras.applications.densenet import DenseNet121

In [ ]:
# Load pretrained DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, pooling='avg', input_shape=(256, 256, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add new classification head
flat1 = Flatten()(base_model.output)
num_classes = 46
output = Dense(num_classes, activation='softmax')(flat1)

# Define model
model = Model(inputs=base_model.inputs, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy'])

# Callbacks
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]



In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator),
    callbacks=callbacks
)

2024-12-01 19:35:00.627683: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-12-01 19:35:00.633300: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


32/32 [==============================] - ETA: 0s - loss: 4.1861 - accuracy: 0.0207

2024-12-01 19:38:14.475305: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


32/32 [==============================] - 362s 11s/step - loss: 4.1861 - accuracy: 0.0207 - val_loss: 3.9995 - val_accuracy: 0.0322 - lr: 1.0000e-04
